In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("XML to DataFrame").getOrCreate()

In [0]:
import requests
import xml.etree.ElementTree as ET

In [0]:
collections = ['sample3', 'sample2', 'sample']
collection = []
for i in collections:
    collection.append(i)

In [0]:
collection

['sample3', 'sample2', 'sample']

In [0]:
# Define the URL to fetch XML data
for i in collections:
    url = f'https://1d55-2001-4490-4e9c-990e-bc6a-bb57-3bac-cea5.ngrok-free.app/{i}'
    response = requests.get(url)

    if response.status_code == 200:
        try:
            root = ET.fromstring(response.content)
            data = ET.tostring(root, encoding = 'utf8').decode('utf8')
            #print(data)
        except ET.ParseError as e:
            print(f'Error parsing XML :  {e}')
    else:
        print(f'Failed tp fetch data :  {response.status_code}')

    #Storing all xml data in the dbfs storage 
    output = f'dbfs:/FileStore/tables/{i}.xml'
    dbutils.fs.put(output, data, overwrite = True)

Wrote 3342 bytes.
Wrote 3429 bytes.
Wrote 697 bytes.


In [0]:
%fs ls /FileStore/tables/


path,name,size,modificationTime
dbfs:/FileStore/tables/SAMPLE.zip,SAMPLE.zip,6624,1725622415000
dbfs:/FileStore/tables/authors.xml,authors.xml,1687,1725605601000
dbfs:/FileStore/tables/config-1.json,config-1.json,276,1723611879000
dbfs:/FileStore/tables/config.json,config.json,276,1723611517000
dbfs:/FileStore/tables/sample.xml,sample.xml,1153,1728966407000
dbfs:/FileStore/tables/sample2.xml,sample2.xml,3473,1729147307000
dbfs:/FileStore/tables/sample3.xml,sample3.xml,3386,1729147307000
dbfs:/FileStore/tables/test.txt,test.txt,47,1724915571000


In [0]:
tag = {
    'sample' : 'customers',
    'sample2' : 'company',
    'sample3' : 'employees',
}

for i, row_tag in tag.items():
    df = spark.read.format('xml').option('rowTag', row_tag).load(f'dbfs:/FileStore/tables/{i}.xml') 
    print(f'DataFrame of {i}')
    display(df)

DataFrame of sample


customer
"List(List(List(New York, Manhattan, NY), 30, 1, John Doe, List(List(List(1001, Book, 2), List(1002, Pen, 10)))), List(List(Los Angeles, Hollywood, CA), 25, 2, Jane Smith, List(List(List(1003, Laptop, 1)))))"


DataFrame of sample2


departments,location,name,products
"List(List(List(List(List(List(null, List(alice@techcorp.com, +1-555-1010), 10, 1001, Alice, List(List(Project X, Project Z)), Senior Software Engineer, 150000, List(List(Java, Python, C++)), null), List(null, List(bob@techcorp.com, +1-555-2020), 7, 1002, Bob, List(List(Project Y)), Data Scientist, 120000, List(List(Python, R, SQL)), null))), Engineering), List(List(List(List(null, List(carol@techcorp.com, +1-555-3030), 12, 2001, Carol, null, Sales Manager, 90000, null, List(List(USA, Canada))))), Sales), List(List(List(List(List(List(List(250000, Brand Awareness 2024, Ongoing), List(500000, Product Launch 2023, Completed))), List(david@techcorp.com, +1-555-4040), 5, 3001, David, null, Marketing Specialist, 85000, null, null))), Marketing)))","List(List(List(List(New York, 300, List(List(List(500000, Project X, Ongoing), List(1200000, Project Y, Completed)))), List(Chicago, 150, List(List(List(800000, Project Z, Planning)))))), List(789 Innovation Drive, San Francisco, USA, CA, 94107))",TechCorp,"List(List(List(Hardware, List(12 hours, Black, null, null, 10x15 cm, null), P001, 2022-05-01, TechWidget, 250, List(List(List(North America, 1250000, 5000), List(Europe, 750000, 3000)))), List(Software, List(null, null, 1 year, List(List(Windows, macOS, Linux)), null, 2.1), P002, 2023-01-15, TechGadget, 499, List(List(List(North America, 998000, 2000), List(Asia, 748500, 1500))))))"


DataFrame of sample3


employee
"List(List(1, List(New York, USA, 123 Main St, 10001), 30, Engineering, 2015-03-15, 2021-06-01, John Doe, Senior Software Engineer, List(List(List(500000, Project Alpha, Ongoing), List(300000, Project Beta, Completed))), 120000, List(List(Python, SQL, Java, AWS))), List(2, List(San Francisco, USA, 456 Market St, 94107), 28, Marketing, 2018-09-01, 2022-01-15, Jane Smith, Marketing Manager, List(List(List(120000, Campaign 2023, Completed), List(150000, Campaign 2024, Ongoing))), 95000, List(List(SEO, Content Strategy, Google Analytics))), List(3, List(Chicago, USA, 789 Wall St, 60605), 35, Finance, 2010-05-20, 2020-02-10, Alice Johnson, Finance Manager, List(List(List(200000, Budget Forecast 2023, Completed))), 130000, List(List(Financial Analysis, Risk Management, Excel, SAP))), List(4, List(Los Angeles, USA, 101 HR Lane, 90001), 42, Human Resources, 2008-08-25, 2019-09-05, Bob Williams, HR Director, List(List(List(50000, Employee Engagement, Ongoing))), 140000, List(List(Employee Relations, Recruitment, Training, HR Compliance))), List(5, List(Austin, USA, 102 Tech Park, 73301), 38, IT, 2012-11-11, 2020-07-21, Charlie Brown, IT Manager, List(List(List(250000, Network Overhaul, Completed), List(400000, Cloud Migration, Ongoing))), 125000, List(List(Networking, System Administration, Linux, Cybersecurity))))"
